In [1]:
! pip install keras

    100% |████████████████████████████████| 327kB 2.3MB/s 


In [0]:
# FastTest
!wget https://s3-us-west-1.amazonaws.com/fasttext-vectors/crawl-300d-2M.vec.zip
!unzip crawl-300d-2M.vec.zip -d crawl-300d-2M.vec/

# Twitter
# !wget http://nlp.stanford.edu/data/glove.twitter.27B.zip -c
# !unzip glove.twitter.27B.zip -d glove.twitter.27B.200d/

import numpy as np
np.random.seed(32)
import pandas as pd

! pip install pydrive
# these classes allow you to request the Google drive API
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# https://drive.google.com/open?id=1Rk9F9YUMuG9JtuhCaUck1Lxid3OISc1-
file_id = '1Rk9F9YUMuG9JtuhCaUck1Lxid3OISc1-'
downloaded = drive.CreateFile({'id': file_id})
# allows you to temporarily load your file in the notebook VM

downloaded.GetContentFile('train.csv')
train = pd.read_csv('train.csv')

# https://drive.google.com/open?id=1CH6MLJYHK6rtC-p_4kK7Ms17CIisowGZ
file_id = '1CH6MLJYHK6rtC-p_4kK7Ms17CIisowGZ'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('test.csv')
test = pd.read_csv('test.csv')

In [0]:
embed_size = 300
max_features = 100000 
max_text_len = 150

# EMBEDDING_FILE = "glove.840B.300d/glove.840B.300d.txt"
# EMBEDDING_FILE = "glove.twitter.27B.200d/glove.twitter.27B.200d.txt"
EMBEDDING_FILE = "crawl-300d-2M.vec/crawl-300d-2M.vec"

In [4]:
import sys, os, re, csv, codecs, gc
os.environ["OMP_NUM_THREADS"] = "4"
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D
from keras.layers import Bidirectional, Add, Flatten
from keras.models import Model, load_model
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras import backend as K
from keras.engine import InputSpec, Layer

import logging
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback

class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch + 1, score))

Using TensorFlow backend.


In [0]:
def normalize(text):
    text = text.replace('&', ' and ')
    text = text.replace('@', ' at ')
    text = text.replace('0', ' zero ')
    text = text.replace('1', ' one ')
    text = text.replace('2', ' two ')
    text = text.replace('3', ' three ')
    text = text.replace('4', ' four ')
    text = text.replace('5', ' five ')
    text = text.replace('6', ' six ')
    text = text.replace('7', ' seven ')
    text = text.replace('8', ' eight ')
    text = text.replace('9', ' nine ')
    return text

In [10]:
import nltk
nltk.download('punkt')

def clean_corpus(comment):
    comment = comment.lower()
    text = nltk.word_tokenize(comment)
    new_text = " ".join(word for word in text)
    return new_text

[nltk_data] Downloading package punkt to /content/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
category = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train["comment_text"].fillna("no comment", inplace = True)
train["comment_text"] = train["comment_text"].apply(lambda x: normalize(x))
train["comment_text"] = train["comment_text"].apply(lambda x: clean_corpus(x))

test["comment_text"].fillna("no comment", inplace = True)
test["comment_text"] = test["comment_text"].apply(lambda x: normalize(x))
test["comment_text"] = test["comment_text"].apply(lambda x: clean_corpus(x))

In [12]:
nrow_train = train.shape[0]
Y_train = train[category].values
X_train, X_valid, Y_train, Y_valid = train_test_split(train, Y_train, test_size = 0.1)

raw_text_train = X_train["comment_text"].str.lower()
raw_text_valid = X_valid["comment_text"].str.lower()
raw_text_test = test["comment_text"].str.lower()

tk = Tokenizer(num_words = max_features, lower = True)
tk.fit_on_texts(raw_text_train)
X_train["comment_seq"] = tk.texts_to_sequences(raw_text_train.str.lower())
X_valid["comment_seq"] = tk.texts_to_sequences(raw_text_valid.str.lower())
test["comment_seq"] = tk.texts_to_sequences(raw_text_test.str.lower())

X_train = pad_sequences(X_train.comment_seq, maxlen = max_text_len)
X_valid = pad_sequences(X_valid.comment_seq, maxlen = max_text_len)
X_test = pad_sequences(test.comment_seq, maxlen = max_text_len)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [0]:
def get_coefs(word, *arr): return word, np.asarray(arr, dtype = "float32")
embeddings_index = dict(get_coefs(*o.strip().split(" ")) for o in open(EMBEDDING_FILE))

In [0]:
word_index = tk.word_index
nb_words = min(max_features, len(word_index))
# embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [0]:
class AttentionWeightedAverage(Layer):
    """
    Computes a weighted average of the different channels across timesteps.
    Uses 1 parameter pr. channel to compute the attention value for a single timestep.
    """

    def __init__(self, return_attention=False, **kwargs):
        self.init = initializers.get('uniform')
        self.supports_masking = True
        self.return_attention = return_attention
        super(AttentionWeightedAverage, self).__init__(**kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(ndim=3)]
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[2], 1),
                                 name='{}_W'.format(self.name),
                                 initializer=self.init)
        self.trainable_weights = [self.W]
        super(AttentionWeightedAverage, self).build(input_shape)

    def call(self, x, mask=None):
        # computes a probability distribution over the timesteps
        # uses 'max trick' for numerical stability
        # reshape is done to avoid issue with Tensorflow
        # and 1-dimensional weights
        logits = K.dot(x, self.W)
        x_shape = K.shape(x)
        logits = K.reshape(logits, (x_shape[0], x_shape[1]))
        ai = K.exp(logits - K.max(logits, axis=-1, keepdims=True))

        # masked timesteps have zero weight
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            ai = ai * mask
        att_weights = ai / (K.sum(ai, axis=1, keepdims=True) + K.epsilon())
        weighted_input = x * K.expand_dims(att_weights)
        result = K.sum(weighted_input, axis=1)
        if self.return_attention:
            return [result, att_weights]
        return result

    def get_output_shape_for(self, input_shape):
        return self.compute_output_shape(input_shape)

    def compute_output_shape(self, input_shape):
        output_len = input_shape[2]
        if self.return_attention:
            return [(input_shape[0], output_len), (input_shape[0], input_shape[1])]
        return (input_shape[0], output_len)

    def compute_mask(self, input, input_mask=None):
        if isinstance(input_mask, list):
            return [None] * len(input_mask)
        else:
            return None


def pair_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.int32)
    parts = tf.dynamic_partition(y_pred, y_true, 2)
    y_pos = parts[1]
    y_neg = parts[0]
    y_pos = tf.expand_dims(y_pos, 0)
    y_neg = tf.expand_dims(y_neg, -1)
    out = K.sigmoid(y_neg - y_pos)
    return K.mean(out)

In [0]:
from keras.optimizers import Adam, RMSprop
from keras.layers import SpatialDropout1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.layers import concatenate, GRU, CuDNNGRU, CuDNNLSTM, MaxPooling1D, TimeDistributed
from keras.layers.normalization import BatchNormalization

def build_model(units = 0, dr = 0.0, lr_i = 0.0, lr_f = 0.0):
  inp = Input(shape = (max_text_len,))
  main = Embedding(nb_words, embed_size, weights = [embedding_matrix], 
                   input_length = max_text_len, trainable = False)(inp)
  
  main = SpatialDropout1D(dr)(main)
  main = Bidirectional(CuDNNLSTM(units, return_sequences = True))(main)
  main = Conv1D(64, kernel_size = 3, padding = "same", kernel_initializer = "he_uniform", activation = "elu")(main)
  atten = AttentionWeightedAverage()(main)
  max_pool = GlobalMaxPooling1D()(main)
  main = concatenate([atten, max_pool])
  
  out_put = Dense(6, activation = "sigmoid")(main)
  model = Model(inputs = inp, outputs = out_put)
  model.compile(loss = "binary_crossentropy", optimizer = Adam(lr = lr_i, decay = lr_f), metrics = ['accuracy'])
  return model

In [40]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import KFold

early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)

batch_sizes = 128
epochs = 30
units = 128
dr = 0.3
lr_i, lr_f = 1e-3, 0

file_path = "best_model.hdf5"
model = build_model(units = units, dr = dr, lr_i = lr_i)
check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,
                              save_best_only = True, mode = "min", save_weights_only = True)
ra_val = RocAucEvaluation(validation_data = (X_valid, Y_valid), interval = 1)
history = model.fit(X_train, Y_train, batch_size = batch_sizes, epochs = epochs, validation_data = (X_valid, Y_valid), 
                    verbose = 2, callbacks = [check_point, early_stop, ra_val])
model.load_weights(file_path)
pred = model.predict(X_test, batch_size = batch_sizes, verbose = 1)

# n_folds = 5
# cv_scores = []
# pred = 0

# kfold = KFold(n_splits = n_folds, shuffle = True, random_state = 32)
# for i, (train_idx, valid_idx) in enumerate(kfold.split(X_train)):
#   print("\nRunning fold {}/{}".format(i + 1, n_folds))
#   model = None
#   model = build_LSTM_model(units = units, dr = dr, lr_i = lr_i, lr_f = lr_f)
  
#   x_train, y_train = X_train[train_idx], Y_train[train_idx] 
#   x_valid, y_valid = X_train[valid_idx], Y_train[valid_idx]
  
#   ra_val = RocAucEvaluation(validation_data = (x_valid, y_valid), interval = 1)

#   file_path = "fold " + str(i+1) + " best_model.hdf5"
#   check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,
#                                 save_best_only = True, mode = "min", save_weights_only = True)
  
#   history = model.fit(x_train, y_train, batch_size = batch_sizes, epochs = epochs, validation_data = (x_valid, y_valid), 
#                       verbose = 2, callbacks = [check_point, early_stop, ra_val])
#   model.load_weights(file_path)
#   pred += model.predict(X_test, batch_size = batch_sizes, verbose = 1)
  
#   oof_pred = model.predict(x_valid, batch_size = batch_sizes, verbose = 1)
#   cv_score = roc_auc_score(y_valid, oof_pred)
#   cv_scores.append(cv_score)
  
#   del file_path, model
#   gc.collect()
  
# pred = pred/n_flods
# print("\nAverage cv score of {} folds is {}.".format(n_folds, np.mean(cv_scores)))

Train on 143613 samples, validate on 15958 samples
Epoch 1/30
 - 110s - loss: 0.0612 - acc: 0.9787 - val_loss: 0.0427 - val_acc: 0.9836

Epoch 00001: val_loss improved from inf to 0.04268, saving model to best_model.hdf5

 ROC-AUC - epoch: 1 - score: 0.987325
Epoch 2/30
 - 108s - loss: 0.0442 - acc: 0.9831 - val_loss: 0.0395 - val_acc: 0.9844

Epoch 00002: val_loss improved from 0.04268 to 0.03946, saving model to best_model.hdf5

 ROC-AUC - epoch: 2 - score: 0.990321
Epoch 3/30
 - 108s - loss: 0.0417 - acc: 0.9839 - val_loss: 0.0389 - val_acc: 0.9848

Epoch 00003: val_loss improved from 0.03946 to 0.03887, saving model to best_model.hdf5

 ROC-AUC - epoch: 3 - score: 0.991020
Epoch 4/30
 - 108s - loss: 0.0395 - acc: 0.9847 - val_loss: 0.0382 - val_acc: 0.9848

Epoch 00004: val_loss improved from 0.03887 to 0.03823, saving model to best_model.hdf5

 ROC-AUC - epoch: 4 - score: 0.991418
Epoch 5/30
 - 108s - loss: 0.0379 - acc: 0.9854 - val_loss: 0.0392 - val_acc: 0.9848

Epoch 00005: va

153164/153164 [==============================] - 41s 268us/step


In [0]:
submission = pd.DataFrame()
submission = submission.reindex(columns = ["id"] + category)
submission["id"]= test[["id"]]
submission[category[0:6]] = lstm_pred
submission.to_csv("submission.csv", index = False)

In [42]:
submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.998902,0.376713,0.982758,0.130508,0.936714,0.549623
1,0000247867823ef7,0.000755,0.000037,0.000466,0.000040,0.000245,0.000090
2,00013b17ad220c46,0.001845,0.000129,0.001026,0.000109,0.000440,0.000446
3,00017563c3f7919a,0.000440,0.000028,0.000292,0.000042,0.000144,0.000017
4,00017695ad8997eb,0.006562,0.000128,0.000927,0.000261,0.000601,0.000224
